In [1]:
# importamos las librerias necesarias y las configuraciones respectivas para evitar warnings innecesarios
import pandas as pd
import datetime as dt
pd.options.mode.chained_assignment = None

# importamos el archivo con los valores de indicadores en bruto
df = pd.read_csv('indicadoreservicios.csv')
df.head(2)  

periodo_a_evaluar = 2022

In [2]:
# Regularizacion de municipios de parral y dalcahue
df.loc[df['id_Comp_Matriz'] == 7299, 'id_Comp_Matriz'] = 94459

In [3]:
import os
cwd = os.getcwd()
cwd = cwd.replace('ETL','')

In [4]:
df_figem = pd.read_excel(cwd +'Ranking\\FIGEM_2021_SINIM.xlsx', skiprows=23)
df_figem = df_figem[['CODIGO', 'MUNICIPIO', 'Agrupación']]
df_figem .drop(345, axis=0, inplace=True)

In [5]:
dfcodes = pd.read_excel(cwd +'Ranking\\municipios.xlsx')
df_mix = df_figem.merge(dfcodes, how = 'left', left_on='MUNICIPIO', right_on='Nombre subdere')
dfmix = df_mix[['CodMunChC', 'Clasificacion FIGEM']]
dfmix.head(2)

,CodMunChC,Clasificacion FIGEM
0,88313,Grandes comunas metropolitanas con alto y/o me...
1,86571,Grandes comunas metropolitanas con alto y/o me...


In [6]:
df = df.merge(dfmix, how = 'left', left_on='id_Comp_Matriz', right_on='CodMunChC')
df.drop('CodMunChC', axis=1, inplace=True)
df

,id_Comp_Matriz,Servicio,Ministerio,Anio,indicador1,indicador2,indicador3,indicador4,indicador5,indicador6,...,indicador32_III,porComp,porTra,porAnom,porProm,score,Ministerio_y,Monto,Cantidad_licitaciones,Clasificacion FIGEM
0,6957,Central De Abastecimiento Del Sistema Nacional...,Ministerio de Salud,2021,27.3,3.0,0.0,18.319783,27.533875,34.257204,...,1.0,20.0,0.0,0.0,10.0,68.557319,NaN,1.164593e+12,1863,NaN
1,6957,Central De Abastecimiento Del Sistema Nacional...,Ministerio de Salud,2022,26.4,3.0,0.0,22.916667,19.791667,27.831023,...,1.0,27.0,0.0,0.0,13.0,72.150115,NaN,9.635693e+11,1560,NaN
2,7002,Junta Nacional De Auxilio Escolar Y Beca,Ministerio de Educación,2022,31.5,3.0,-1.0,53.582001,15.917160,9.265034,...,1.0,20.0,0.0,0.0,10.0,65.372538,NaN,8.454999e+11,2027,NaN
3,6954,Fondo Nacional De Salud,Ministerio de Salud,2022,21.4,4.0,-4.0,49.709302,37.015504,20.514694,...,1.0,27.0,0.0,10.0,17.0,61.566277,NaN,8.269443e+11,1112,NaN
4,7002,Junta Nacional De Auxilio Escolar Y Beca,Ministerio de Educación,2021,25.9,4.0,0.0,53.065395,14.850136,16.708540,...,1.0,13.0,0.0,0.0,6.0,68.557774,NaN,7.959965e+11,1818,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852,1820894,Corporacion Municipal De Desarrollo Economico ...,Otros,2022,NaN,NaN,NaN,100.000000,0.000000,0.000000,...,NaN,11.0,NaN,NaN,11.0,37.026112,NaN,1.285144e+06,6,NaN
1853,1317359,Asociación Chilena De Municipalidades,Otros,2022,NaN,NaN,NaN,100.000000,0.000000,0.000000,...,NaN,67.0,NaN,NaN,67.0,34.196438,NaN,7.413560e+05,1,NaN
1854,1818087,Corporación Municipal De Desarrollo Social De ...,Otros,2022,NaN,NaN,NaN,100.000000,0.000000,0.000000,...,NaN,67.0,NaN,NaN,67.0,34.196438,NaN,3.817160e+05,1,NaN
1855,7493,Empresa Portuaria Iquique,Otros,2022,0.0,4.0,2.0,100.000000,0.000000,0.000000,...,NaN,60.0,0.0,12.0,40.0,55.386431,NaN,1.917420e+05,1,NaN


In [7]:
# Definimos la función que hará el rankeo de los indicadores de cada servicio
def ranking_indicadores(df, anio):
    # Creamos una lista vacía para guardar los nombres de los indicadores
    l_ind = []
    for i in range(32):
        if i != 6:
            l_ind.append(f'indicador{i+1}')

    # Creamos una lista vacía para guardar los nombres de los indicadores rankeados
    l_rank = []
    for i in range(32):
        if i != 6:
            l_rank.append(f'indicador{i+1}_r')

    # Filtramos el df por año
    dft = df[df['Anio'] == anio]

    # Ciclo para recorrer cada indicador y calcular su ranking
    for i in range(31):
        if l_ind[i] in ['indicador2', 'indicador3', 'indicador4', 'indicador12', 'indicador13', 'indicador27']:
            dft[l_rank[i]] = dft[l_ind[i]].rank(method='min', ascending=False)
        else:
            dft[l_rank[i]] = dft[l_ind[i]].rank(method='min', ascending=True)
    
    # sub df con los valores de los indicadores rankeados para sacar promedio y calcualr nuevo rankling en base a ese promedio
    dft2 = dft[['id_Comp_Matriz', 'Clasificacion FIGEM', 'indicador1_r', 'indicador2_r', 'indicador3_r', 'indicador4_r', 'indicador5_r', 'indicador6_r', 'indicador8_r', 'indicador9_r', 'indicador10_r', 'indicador11_r', 'indicador12_r', 'indicador13_r', 'indicador14_r', 'indicador15_r', 'indicador16_r', 'indicador17_r', 'indicador18_r', 'indicador19_r', 'indicador20_r', 'indicador21_r', 'indicador22_r', 'indicador23_r', 'indicador24_r', 'indicador25_r', 'indicador26_r', 'indicador27_r', 'indicador28_r', 'indicador29_r', 'indicador30_r', 'indicador31_r', 'indicador32_r']]
    dft2['total_rank_mean'] = dft2.loc[:, 'indicador1_r':'indicador32_r'].mean(axis=1)
    dft2['final_rank'] = dft2['total_rank_mean'].rank(method='min')
    dft2 = dft2[['id_Comp_Matriz', 'total_rank_mean', 'final_rank']]
    
    # Unimo los dfs para obtener un df con los valores de los indicadores rankeados totales
    dffinal = dft.merge(dft2, on='id_Comp_Matriz')

    return dffinal


In [8]:
# Calculo para el Sector Municipal 
df_mun = df[df['Ministerio']=='Municipios']
df_mun_rank = pd.DataFrame()
for i in df_mun['Clasificacion FIGEM'].unique():
    df_mun_filt = df_mun[df_mun['Clasificacion FIGEM']==i]
    df_mun_rank = pd.concat([df_mun_rank, ranking_indicadores(df_mun_filt, periodo_a_evaluar)], axis=0)

df_mun_rank.head(5).sort_values(by='final_rank', ascending=True)

,id_Comp_Matriz,Servicio,Ministerio,Anio,indicador1,indicador2,indicador3,indicador4,indicador5,indicador6,...,indicador25_r,indicador26_r,indicador27_r,indicador28_r,indicador29_r,indicador30_r,indicador31_r,indicador32_r,total_rank_mean,final_rank
0,94459,I Municipalidad De Parral,Municipios,2022,45.8,3.0,1.0,38.826691,13.121361,15.168546,...,1.0,13.0,3.0,1.0,1.0,48.0,40.0,NaN,20.066667,13.0
2,94459,Departamento De Salud Parral,Municipios,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.066667,13.0
4,99860,Municipalidad De Ovalle,Municipios,2022,36.9,3.0,0.0,26.406353,4.301787,4.456435,...,1.0,21.0,17.0,18.0,1.0,31.0,23.0,1.0,22.612903,22.0
1,94459,I Municipalidad De Parral,Municipios,2022,45.8,3.0,1.0,38.826691,13.121361,15.168546,...,1.0,13.0,3.0,1.0,1.0,48.0,40.0,NaN,NaN,NaN
3,94459,Departamento De Salud Parral,Municipios,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_mun_rank

,id_Comp_Matriz,Servicio,Ministerio,Anio,indicador1,indicador2,indicador3,indicador4,indicador5,indicador6,...,indicador25_r,indicador26_r,indicador27_r,indicador28_r,indicador29_r,indicador30_r,indicador31_r,indicador32_r,total_rank_mean,final_rank
0,94459,I Municipalidad De Parral,Municipios,2022,45.8,3.0,1.0,38.826691,13.121361,15.168546,...,1.0,13.0,3.0,1.0,1.0,48.0,40.0,NaN,20.066667,13.0
1,94459,I Municipalidad De Parral,Municipios,2022,45.8,3.0,1.0,38.826691,13.121361,15.168546,...,1.0,13.0,3.0,1.0,1.0,48.0,40.0,NaN,NaN,NaN
2,94459,Departamento De Salud Parral,Municipios,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.066667,13.0
3,94459,Departamento De Salud Parral,Municipios,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99860,Municipalidad De Ovalle,Municipios,2022,36.9,3.0,0.0,26.406353,4.301787,4.456435,...,1.0,21.0,17.0,18.0,1.0,31.0,23.0,1.0,22.612903,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,123529,I Municipalidad De Lago Verde,Municipios,2022,55.6,2.0,1.0,43.715847,9.016393,6.308770,...,1.0,83.0,57.0,1.0,1.0,56.0,72.0,1.0,50.900000,73.0
105,119999,I Municipalidad De Laguna Blanca,Municipios,2022,50.0,2.0,-1.0,58.928571,32.142857,16.445177,...,1.0,5.0,109.0,1.0,1.0,89.0,81.0,NaN,53.900000,85.0
106,165107,I Municipalidad De Las Guaitecas,Municipios,2022,59.3,3.0,1.0,48.283753,27.917620,28.809555,...,1.0,83.0,16.0,55.0,1.0,101.0,104.0,NaN,60.433333,106.0
107,116403,I Municipalidad De Puqueldon,Municipios,2022,86.7,1.0,0.0,41.780822,38.698630,26.039336,...,104.0,83.0,57.0,51.0,1.0,1.0,23.0,NaN,62.785714,107.0


In [10]:
# Calculo para el Sector Administración Central 
df_ac = df[df['Ministerio'].str.lower().str.contains('ministerio')]
df_ac = df_ac[~df_ac['Servicio'].str.lower().str.contains('público')]
df_ac_rank = ranking_indicadores(df_ac, periodo_a_evaluar)
df_ac_rank.head(5).sort_values(by='final_rank', ascending=True)

,id_Comp_Matriz,Servicio,Ministerio,Anio,indicador1,indicador2,indicador3,indicador4,indicador5,indicador6,...,indicador25_r,indicador26_r,indicador27_r,indicador28_r,indicador29_r,indicador30_r,indicador31_r,indicador32_r,total_rank_mean,final_rank
0,6957,Central De Abastecimiento Del Sistema Nacional...,Ministerio de Salud,2022,26.4,3.0,0.0,22.916667,19.791667,27.831023,...,1.0,1.0,252.0,197.0,NaN,100.0,97.0,1.0,138.100000,29.0
1,7002,Junta Nacional De Auxilio Escolar Y Beca,Ministerio de Educación,2022,31.5,3.0,-1.0,53.582001,15.917160,9.265034,...,1.0,121.0,141.0,232.0,1.0,156.0,193.0,1.0,171.096774,177.0
2,6954,Fondo Nacional De Salud,Ministerio de Salud,2022,21.4,4.0,-4.0,49.709302,37.015504,20.514694,...,1.0,273.0,363.0,242.0,1.0,142.0,1.0,1.0,187.290323,266.0
3,7243,Ministerio De Obras Publicas Direc Cion Gral D...,Ministerio de Obras Públicas,2022,20.8,4.0,1.0,34.405458,17.543860,11.802625,...,1.0,136.0,390.0,206.0,1.0,273.0,316.0,NaN,207.500000,365.0
4,113812,Direccion De Logistica De Carabineros,Ministerio del interior y Seguridad pública,2022,36.2,3.0,0.0,16.263806,7.337166,49.886704,...,447.0,249.0,231.0,281.0,1.0,317.0,218.0,1.0,236.935484,430.0


In [11]:
df_mun_rank.sort_values(by='final_rank').head(5)

,id_Comp_Matriz,Servicio,Ministerio,Anio,indicador1,indicador2,indicador3,indicador4,indicador5,indicador6,...,indicador25_r,indicador26_r,indicador27_r,indicador28_r,indicador29_r,indicador30_r,indicador31_r,indicador32_r,total_rank_mean,final_rank
8,116296,I Municipalidad De Las Cabras,Municipios,2022,50.0,4.0,0.0,69.535284,23.883162,13.157374,...,1.0,5.0,30.0,75.0,1.0,6.0,13.0,1.0,25.161290,1.0
11,93774,I Municipalidad De San Clemente,Municipios,2022,17.3,7.0,1.0,63.765182,26.214575,9.383585,...,1.0,39.0,20.0,79.0,1.0,5.0,1.0,NaN,26.233333,1.0
7,87901,I Municipalidad De Coquimbo,Municipios,2022,23.6,4.0,0.0,46.338606,5.741066,4.017665,...,1.0,14.0,8.0,5.0,1.0,9.0,18.0,1.0,11.258065,1.0
13,100107,Ilustre Municipalidad De San Carlos,Municipios,2022,34.9,4.0,1.0,29.754033,16.715155,18.034831,...,1.0,17.0,9.0,17.0,1.0,5.0,17.0,1.0,13.548387,1.0
20,7325,I Municipalidad De Recoleta,Municipios,2022,28.8,4.0,0.0,30.677291,6.146841,23.084317,...,1.0,5.0,10.0,10.0,1.0,1.0,17.0,1.0,14.066667,1.0


In [12]:
dfmun1 = df_mun_rank[['id_Comp_Matriz', 'Servicio',  'Ministerio',  'Anio',  'indicador1_r',  'indicador2_r', 'indicador3_r', 'indicador4_r', 'indicador5_r', 'indicador6_r', 'indicador8_r', 'indicador9_r', 'indicador10_r', 'indicador11_r', 'indicador12_r', 'indicador13_r', 'indicador14_r', 'indicador15_r', 'indicador16_r', 'indicador17_r', 'indicador18_r', 'indicador19_r', 'indicador20_r', 'indicador21_r', 'indicador22_r', 'indicador23_r', 'indicador24_r', 'indicador25_r', 'indicador26_r', 'indicador27_r', 'indicador28_r', 'indicador29_r', 'indicador30_r', 'indicador31_r', 'indicador32_r', 'final_rank', 'total_rank_mean', 'Clasificacion FIGEM']]
dfmun2 = df_mun_rank[['id_Comp_Matriz', 'Servicio',  'Ministerio',  'Anio',  'indicador1',  'indicador2', 'indicador3', 'indicador4', 'indicador5', 'indicador6', 'indicador8', 'indicador9', 'indicador10', 'indicador11', 'indicador12', 'indicador13', 'indicador14', 'indicador15', 'indicador16', 'indicador17', 'indicador18', 'indicador19', 'indicador20', 'indicador21', 'indicador22', 'indicador23', 'indicador24', 'indicador25', 'indicador26', 'indicador27', 'indicador28', 'indicador29', 'indicador30', 'indicador31', 'indicador32', 'final_rank', 'total_rank_mean', 'Clasificacion FIGEM']]

dfmunmelted1 = pd.melt(dfmun1, id_vars=['id_Comp_Matriz',	'Servicio',	'Ministerio',	'Anio', 'final_rank', 'total_rank_mean', 'Clasificacion FIGEM'])
dfmunmelted2 = pd.melt(dfmun2, id_vars=['id_Comp_Matriz',	'Servicio',	'Ministerio',	'Anio', 'final_rank', 'total_rank_mean', 'Clasificacion FIGEM'])

dfmunmelted1['variable'] = dfmunmelted1['variable'].str.replace('_r', '')

dfmunmelted = dfmunmelted1.merge(dfmunmelted2, on=['id_Comp_Matriz',	'Servicio',	'Ministerio',	'Anio', 'final_rank', 'total_rank_mean', 'variable', 'Clasificacion FIGEM'])

dfmunmelted.rename(columns={'value_x': 'Ranking Indicador', 'value_y':'Valor Indicador'}, inplace=True)

In [13]:
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador1', 'variable'] = 'Porcentaje de licitaciones públicas y privadas con un único oferente'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador2', 'variable'] = 'Promedio de participantes por procedimiento'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador3', 'variable'] = 'Cambio en el promedio de participantes por procedimiento'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador4', 'variable'] = 'Número de empresas ganadoras diferentes entre todos los contratos'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador5', 'variable'] = 'Porcentaje de procedimientos que utilizaron adjudicación directa o licitación privada'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador6', 'variable'] = 'Porcentaje del monto adjudicado por adjudicación directa o licitación privada'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador8', 'variable'] = 'IHH por número de contratos'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador9', 'variable'] = 'IHH por monto'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador10', 'variable'] = 'ID por número de contratos'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador11', 'variable'] = 'ID por monto'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador12', 'variable'] = 'Índice de participación'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador13', 'variable'] = 'Número de participantes distintos entre todos los procedimientos'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador14', 'variable'] = 'Índice de concentración de las 4 empresas con más procedimientos ganados'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador15', 'variable'] = 'Índice de concentración de las 4 empresas con más monto adjudicado'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador16', 'variable'] = 'Ratio de participación y adjudicación en licitaciones públicas y privadas'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador17', 'variable'] = 'Porcentaje de licitaciones públicas y privadas sin ningún documento obligatorio publicado'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador18', 'variable'] = 'Promedio del porcentaje de incumplimiento en la publicación de documentos obligatorios'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador19', 'variable'] = 'Porcentaje de licitaciones públicas y privadas con varias adjudicaciones'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador20', 'variable'] = 'Porcentaje de las licitaciones mayores o iguales a 100 UTM y menores a 1.000 UTM cuyo plazo de oferta fue acelerado'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador21', 'variable'] = 'Porcentaje de las licitaciones mayores o iguales a 1.000 UTM y menores a 5.000 UTM cuyo plazo de oferta fue acelerado'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador22', 'variable'] = 'Porcentaje de licitaciones sin bases de licitación'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador23', 'variable'] = 'Porcentaje de licitaciones sin respuesta a todas las preguntas del foro'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador24', 'variable'] = 'Porcentaje de procedimientos sin acta de evaluación'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador25', 'variable'] = 'Porcentaje de procedimientos sin resolución de adjudicación'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador26', 'variable'] = 'Porcentaje de procedimientos sin contrato'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador27', 'variable'] = 'Cambio en el porcentaje de contratos publicados por procedimiento'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador28', 'variable'] = 'Porcentaje de licitaciones que no cumplen con la normativa de publicación'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador29', 'variable'] = 'Porcentaje de las licitaciones menores a 100 UTM cuyo plazo de oferta fue menor al plazo mínimo establecido por la ley de compras públicas'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador30', 'variable'] = 'Porcentaje de las licitaciones mayores o iguales a 100 UTM y menores a 1.000 UTM cuyo plazo de oferta fue menor al plazo mínimo establecido por la ley de compras públicas'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador31', 'variable'] = 'Porcentaje de las licitaciones mayores o iguales a 1.000 UTM y menores a 5.000 UTM cuyo plazo de oferta fue menor al plazo mínimo establecido por la ley de compras públicas'
dfmunmelted.loc[dfmunmelted['variable'] == 'indicador32', 'variable'] = 'Porcentaje de las licitaciones mayores o iguales a 5.000 UTM cuyo plazo de oferta fue menor al plazo mínimo establecido por la ley de compras públicas'

In [14]:
dfmunmelted.to_excel('municipios_rank2.xlsx', index=False)

In [15]:
dfmunmelted.sort_values(by='Servicio', ascending=False).head(34)

,id_Comp_Matriz,Servicio,Ministerio,Anio,final_rank,total_rank_mean,Clasificacion FIGEM,variable,Ranking Indicador,Valor Indicador
10475,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de las licitaciones mayores o igual...,NaN,NaN
7691,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de procedimientos sin acta de evalu...,1.0,0.000000
3515,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Índice de participación,15.0,3.396000
3863,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Número de participantes distintos entre todos ...,21.0,2.283133
6647,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de las licitaciones mayores o igual...,38.0,52.940000
8039,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de procedimientos sin resolución de...,1.0,0.000000
5603,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Promedio del porcentaje de incumplimiento en l...,36.0,25.753012
8387,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de procedimientos sin contrato,35.0,96.000000
35,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de licitaciones públicas y privadas...,15.0,32.500000
6299,115163,Municipalidad De Traiguen,Municipios,2022,23.0,23.200000,"Comunas urbanas medianas, con desarrollo medio.",Porcentaje de las licitaciones mayores o igual...,46.0,55.420000
